In [2]:
!pip install xgboost -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install scikit-learn -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 7.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 7.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 MB 7.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 6.1 MB/s eta 0:00:00a 0:00:01


In [3]:
!pip install pandas numpy -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 3.5 MB/s eta 0:00:00a 0:00:01


In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv("./data/predict_table.csv") 
data2 = pd.read_csv("./data/product_info_simple_final_train.csv") 

In [3]:
from datetime import datetime
base = int(datetime.strptime(str(20210104), "%Y%m%d").timestamp())+58320000
change = lambda x:(int(datetime.strptime(str(x), "%Y%m%d").timestamp())-base)/86400

In [6]:
models = {}
models2 = {}
models3 = {}

import xgboost
for i in data['product_pid'].unique():
    X=data2[data2['product_pid']==i]["transaction_date"].apply(lambda x:change(x)).values.reshape(-1,1)
    y=data2[data2['product_pid']==i]["apply_amt"]
    y2=data2[data2['product_pid']==i]["redeem_amt"]
    y3=data2[data2['product_pid']==i]["net_in_amt"]
    models[i]=xgboost.XGBRegressor(tree_method='gpu_hist', gpu_id=1)
    models[i].fit(X,y)
    models2[i]=xgboost.XGBRegressor(tree_method='gpu_hist', gpu_id=2)
    models2[i].fit(X,y2)
    models3[i]=xgboost.XGBRegressor(tree_method='gpu_hist', gpu_id=3)
    models3[i].fit(X,y3)

In [9]:
for i in data["product_pid"].unique():
    X=data[data['product_pid']==i]["transaction_date"].apply(lambda x:change(x)).values.reshape(-1,1)
    pre = models[i].predict(X)
    pre2 = models2[i].predict(X)
    pre3 = models3[i].predict(X)
    data.loc[data['product_pid']==i,"apply_amt_pred"] = pre
    data.loc[data['product_pid']==i,"redeem_amt_pred"] = pre2
    data.loc[data['product_pid']==i,"net_in_amt_pred"] = pre3

In [10]:

data["net_in_amt_pred"] = (data["apply_amt_pred"]-data["redeem_amt_pred"])*0.5+data["net_in_amt_pred"]*0.5
data.to_csv("predict_table.csv",index=False)
data

,product_pid,transaction_date,apply_amt_pred,redeem_amt_pred,net_in_amt_pred
0,product1,20221110,3.360803,0.050494,3.310022
1,product1,20221111,3.360803,0.050494,3.310022
2,product1,20221114,3.360803,0.050494,3.310022
3,product1,20221115,3.360803,0.050494,3.310022
4,product1,20221116,3.360803,0.050494,3.310022
...,...,...,...,...,...
1385,product99,20221117,1.047541,9.708147,-8.663309
1386,product99,20221118,1.047541,9.708147,-8.663309
1387,product99,20221121,1.047541,9.708147,-8.663309
1388,product99,20221122,1.047541,9.708147,-8.663309
